In [36]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, confusion_matrix
from sklearn.dummy import DummyClassifier
from sklearn.pipeline import Pipeline

In [37]:
data = pd.read_csv('/content/drive/MyDrive/Домашка по курсу "Python для анализа данных"/train.csv')

Сначала я хотела сделать акцент именно на том, чтобы не упустить выживших людей (минимизировать ложноотрицательные случаи), поэтому выбрала основной метрикой  Recall, однако модель показала результат 0,75 на тестовой выборке, что меня не устроило. Поэтому решила остановиться на метрике ROC-AUC, изменив цель на то, чтобы спрогнозировать вероятность выживания.

In [38]:
print(data.head())

   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN        S  


In [39]:
print(data['Survived'].value_counts(normalize=True))

Survived
0    0.616162
1    0.383838
Name: proportion, dtype: float64


In [15]:
# Предобработка данных
# Помимо класса, пола и возраста я добавила также кол-во родственников, тк это может влиять на выживание
features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch']
X = data[features]
y = data['Survived']

In [40]:
# Тк в ДС указаны данные о возврасте не всех пассажиров, то заменяем пропуски в Age медианными значениями для каждого класса Pclass
for pclass in [1, 2, 3]:
    median_age = X.loc[X['Pclass'] == pclass, 'Age'].median()
    X.loc[(X['Age'].isnull()) & (X['Pclass'] == pclass), 'Age'] = median_age

In [18]:
X = pd.get_dummies(X, drop_first=True)

In [19]:
# Разбиение на тренировочную, валидационную и тестовую выборки
# 60% - тренировочная, 20% - валидационная, 20% - тестовая
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.25, random_state=42, stratify=y_train_val)


In [20]:
# Бейзлайн модель
# Константное предсказание - случайное предсказание с равномерным распределением
dummy_clf = DummyClassifier(strategy='uniform', random_state=42)
dummy_clf.fit(X_train, y_train)
y_prob_dummy_val = dummy_clf.predict_proba(X_val)[:, 1]  # Вероятности класса 1


In [21]:
# Оценка качества бейзлайна на валидационной выборке
roc_auc_dummy_val = roc_auc_score(y_val, y_prob_dummy_val)

print("Baseline (Dummy Classifier) on Validation Set:")
print(f"ROC-AUC: {roc_auc_dummy_val:.2f}")

Baseline (Dummy Classifier) on Validation Set:
ROC-AUC: 0.50


In [23]:
# ML-модель (Logistic Regression)
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', LogisticRegression(random_state=42, max_iter=1000))
])


In [24]:
# Обучаем модель на тренировочной выборке
pipeline.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('classifier',
                 LogisticRegression(max_iter=1000, random_state=42))])

In [35]:
# Получаем вероятности на валидационной выборке
y_prob_lr_val = pipeline.predict_proba(X_val)[:, 1]  # Вероятности класса 1

In [30]:
# Оценка качества ML-модели на валидационной выборке
roc_auc_lr_val = roc_auc_score(y_val, y_prob_lr_val)

print("\nLogistic Regression Model on Validation Set:")
print(f"ROC-AUC: {roc_auc_lr_val:.2f}")


Logistic Regression Model on Validation Set:
ROC-AUC: 0.86


In [28]:
# Вывод матрицы ошибок для анализа (при пороге 0.5)
y_pred_lr_val = (y_prob_lr_val >= 0.5).astype(int)
conf_matrix_val = confusion_matrix(y_val, y_pred_lr_val)
print("Confusion Matrix (Validation):")
print(conf_matrix_val)

Confusion Matrix (Validation):
[[93 17]
 [21 47]]


In [29]:
# Финальная оценка на тестовой выборке
# Получаем вероятности на тестовой выборке
y_prob_lr_test = pipeline.predict_proba(X_test)[:, 1]

In [31]:
# Оценка качества на тестовой выборке
roc_auc_lr_test = roc_auc_score(y_test, y_prob_lr_test)

print("\nLogistic Regression Model on Test Set:")
print(f"ROC-AUC: {roc_auc_lr_test:.2f}")


Logistic Regression Model on Test Set:
ROC-AUC: 0.84


In [32]:
# Вывод матрицы ошибок для тестовой выборки (при пороге 0.5)
y_pred_lr_test = (y_prob_lr_test >= 0.5).astype(int)
conf_matrix_test = confusion_matrix(y_test, y_pred_lr_test)
print("Confusion Matrix (Test):")
print(conf_matrix_test)

Confusion Matrix (Test):
[[96 14]
 [21 48]]
